In [ ]:
import re
from bs4 import BeautifulSoup
import html

In [ ]:
with open('avar_rus_selected.htm', 'r', encoding='utf-8') as f:
    raw_html = html.unescape(f.read())
raw_html = re.sub("\sstyle='[^']+'", '', raw_html)
raw_html = re.sub("\sclass=[A-Za-z]+", '', raw_html)
raw_html = re.sub('<o:p>.?</o:p>', '', raw_html)
raw_html = re.sub('[\xa0\n]', ' ', raw_html)
raw_html = re.sub('</?span>', '', raw_html)
raw_html = re.sub('</b><b>', '', raw_html)
raw_html = re.sub('  +', ' ', raw_html)
raw_html = re.sub('І', "I", raw_html)

raw_paragraphs = re.findall('<p.+?</p>', raw_html, re.DOTALL)
len(raw_paragraphs)

In [ ]:
raw_paragraphs[58]

In [ ]:
lemmas = []
for p in raw_paragraphs:
    if re.search('<b>.*?</b>', p):
        lemmas.append(re.search('<b>(.*?)</b>', p).group(1).strip())
    else:
        lemmas.append('NONE')
len(lemmas)

In [ ]:
lemmas

In [ ]:
with open('avar_rus_paragraphs.htm', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(raw_paragraphs))
with open('avar_rus_lemmas.txt', 'w', encoding='utf-8') as fl:
    fl.write('\n'.join(lemmas))

In [ ]:
with open('avar_rus_lemmas.txt', 'r', encoding='utf-8') as f:
    lemmas_str = f.read()

In [ ]:
from collections import Counter

In [ ]:
letters = ''.join(list(set(lemmas_str)))
re.findall("[^supi!/IА-Яа-я\d\n\-:~\(\)<> ]", letters)

In [ ]:
# trying to clean html-tags once and for all
#[ac-hj-oqrtv-z]

In [ ]:
def multiple_re_sub(string, repl_doc):
    repl_list = [pair.split(',') for pair in repl_doc.split('\n')]
    for pair in repl_list:
        string = re.sub(pair[0], pair[1], string)
    return string

In [ ]:
with open('avar_rus_paragraphs.htm', 'r', encoding='utf-8') as f:
    htm_par = f.read()
    
with open('_repl_technical.txt', 'r', encoding='utf-8') as f_r:
    repl_doc = f_r.read()
    
htm_changed = multiple_re_sub(htm_par, repl_doc)
with open('avar_rus_paragraphs_clean.htm', 'w', encoding='utf-8') as fw:
    fw.write(htm_changed)

In [ ]:
lemmas = []
for p in htm_changed.split('\n'):
    if re.search('<b>.*?</b>', p):
        lemmas.append(re.search('<b>(.*?)</b>', p).group(1).strip())
    else:
        lemmas.append('NONE')
        
with open('avar_rus_lemmas_cleaned.txt', 'w', encoding='utf-8') as fl:
    fl.write('\n'.join(lemmas))

### работа с файлом про леммы:
- вычленить грамматику
- вычленить информацию об омонимах
- фонетизация
- достать чистые заглавные слова
- все грамматические формы (_надо_: уточнить грамматику. окончания, классы)

In [ ]:
import pandas as pd

In [1]:
import re
from collections import OrderedDict
import json

In [108]:
with open('avar_rus_lemmas_cleaned.txt', 'r', encoding='utf-8') as f:
    lemmas = f.read().split('\n')
    print(len(lemmas))
    


22842


In [51]:
test_lemmas = lemmas[:100]
#test_lemmas = []
TO_BE_ADDED = [1422, 1429, 1430, 1617, 1769, 2087, 22680, 22493, 22794, 22804, 793, 796, 797, 1199, 1277, 1321, 6035]
for i in TO_BE_ADDED:
    test_lemmas.append(lemmas[i])

In [109]:
def produce_variants(string, gram, CLASSES = 'бвйр'):
    NUM_slashes = len(re.findall('/', string))
    NUM_brackets = len(re.findall('\(.\)', string))
    
    
    if not gram and NUM_slashes + NUM_brackets == 0:
        answer = [string]
        
    if NUM_brackets:
        sub_answer = []
        for marker in CLASSES:
            sub_answer.append(re.sub('\(.\)', marker, string))
    else:
        sub_answer = [string]
    
    # slash_classing
    if NUM_slashes:
        parts_string = string.split('-')
        IS_FLEX = 0
        GIVEN = ""
        # check if this slashes refer to classes and not grammar
        for part in parts_string:
            if re.match("[бвйрюяе]'?/", part):
                IS_FLEX += 1
                GIVEN = re.search("^([бвйрюяе]'?)/", part).group(1)
            if re.search('/[бвр]$', part) and not gram:
                IS_FLEX = -5
                GIVEN = re.search('/[бвй]', part).group()
                
        # doing replaces
        if GIVEN and IS_FLEX > 0:
            repl = GIVEN + "/"
            new_sub_answer = []
            
            if GIVEN[0] in CLASSES:
                for word in sub_answer:
                    for letter in CLASSES:
                        new_word = re.sub(repl, letter, word)
                        new_sub_answer.append(new_word)
            
            # avar vowels can be class markers, too, idk what monster did this script for them
            else:
                if GIVEN[0] == "е":
                    plus_patt = "е"
                elif GIVEN[0] == "ю":
                    plus_patt = "у"
                elif GIVEN[0] == "я":
                    plus_patt = "а"
                for word in sub_answer:
                    new_sub_answer.append(word)
                    for letter in "бвр":
                        patt = letter + plus_patt
                        new_word = re.sub(repl, patt, word)
                        new_sub_answer.append(new_word)
                        
            sub_answer = new_sub_answer
            
        elif IS_FLEX < 0:
            new_sub_answer = []
            for word in sub_answer:
                for letter in CLASSES:
                    new_word = re.sub(GIVEN, letter, word)
                    new_sub_answer.append(new_word)
                        
            sub_answer = new_sub_answer
    
    if gram:
        grammar = re.sub('//', ',', gram[1:-1])
        gram_list = [x.strip() for x in grammar.split(',')]
        answer = []
        for word in sub_answer:
            answer.append(word)
        for form in gram_list:
            if form.startswith('-'):
                for word in sub_answer:
                    # we suppose here that only grammar-slashes remain
                    if '/' in word:
                        word = word.rsplit('/', maxsplit=1)[0]
                    answer.append(word+form[1:])
            else:
                answer.append(form)
        
    else:
        answer = sub_answer
    
    # finally, we can tidy our words of all the brackets and such
    output = []
    sub_output = []
    for word in answer:
        word = re.sub("[, /\(\)']", '', word)
        output.append(word)
        
        # maybe will delete later: all words with a dash (hyphen) must be found by their parts
        if "-" in word:
            for x in word.split('-'):
                sub_output.append(x)
    output.extend(sub_output)  
        
    return list(OrderedDict.fromkeys(output))

In [113]:
def lemma_list_working(lemma_list, verbose=0):
    _hardcode_1 = "(-алъ, -алъул, го'рда(р)игъал)"
    _hardcode_2 = "(-налъ, -налъул, гьа(р)у'духъаби)"

    l_root = []
    l_grammar = []
    l_omonID = []
    l_variants = [] #list of lists
    l_lemmas = []
    l_base_variant = []
    
    count = 0
    
    for row in lemma_list:
        count += 1
        if verbose > 0:
            if count % verbose == 1:
                print(f"already done:\t{count-1}")
        # вытаскиваем номера омонимов, чистим от оставшихся тэгов
        if "<sup>" in row:
            l_omonID.append(int(re.search('<sup>(\d)</sup>', row).group(1)))
            row = re.sub('<sup>.</sup>', '', row)
        else:
            l_omonID.append(0)
    
        # extract grammar
        ## sadly i had to hardcode it
        if row.strip().endswith(_hardcode_1):
            grammar = _hardcode_1
        elif row.strip().endswith(_hardcode_2):
            grammar = _hardcode_2
    
        elif re.search("\([^\)]*[,\-][^\)]*\)", row):
            grammar = re.search("\([^\)]*[,\-][^\)]*\)", row).group()        
        else:
            grammar = ""
        l_grammar.append(grammar)
        # make row without grammar
        if grammar:
            row = re.sub(grammar, '', row).strip()[:-2].strip()
        
        # making row lemmas as in dict should be
        if ":" in row:
            TO_BE_FLEXED, context = row.split(':')
            TO_BE_FLEXED = TO_BE_FLEXED.strip()
            l_lemmas.append(context.strip().replace('~', TO_BE_FLEXED))
        else:
            TO_BE_FLEXED = row.strip()
            l_lemmas.append(TO_BE_FLEXED)
    
        l_variants.append(produce_variants(TO_BE_FLEXED, grammar))
        l_base_variant.append(re.sub("[, /\(\)']", '', TO_BE_FLEXED))
        
        #trying root
        if '/' in TO_BE_FLEXED:
            _root = TO_BE_FLEXED
            root, ending = _root.rsplit('/', maxsplit=1)
            if len(ending) < len(root):
                l_root.append(re.sub("[, /\(\)']", '', root))
            else:
                l_root.append(re.sub("[, /\(\)']", '', TO_BE_FLEXED))
        else:
            l_root.append(re.sub("[, /\(\)']", '', TO_BE_FLEXED))
        
    return {
            'l_grammar': l_grammar,
            'l_omonID': l_omonID,
            'l_variants': l_variants,
            'l_lemmas': l_lemmas,
            'l_base_variant': l_base_variant,
            'root': l_root
        }



In [114]:
whole_lemmas_dict = lemma_list_working(lemmas, verbose=1000)
whole_gr = whole_lemmas_dict['l_grammar']
whole_om = whole_lemmas_dict['l_omonID']
whole_va = whole_lemmas_dict['l_variants']
whole_le = whole_lemmas_dict['l_lemmas']
whole_ba = whole_lemmas_dict['l_base_variant']
whole_ro = whole_lemmas_dict['root']

already done:	0
already done:	1000
already done:	2000
already done:	3000
already done:	4000
already done:	5000
already done:	6000
already done:	7000
already done:	8000
already done:	9000
already done:	10000
already done:	11000
already done:	12000
already done:	13000
already done:	14000
already done:	15000
already done:	16000
already done:	17000
already done:	18000
already done:	19000
already done:	20000
already done:	21000
already done:	22000


In [81]:
len(lemmas)

22842

In [115]:
len(whole_le)

22842

In [120]:
whole_json = {}
for i in range(len(lemmas)):
    pkey = lemmas[i]
    if pkey not in whole_json:
        key = pkey
    else:
        key = f"{pkey}_{i}"
        
    whole_json[key] = {
        'lemma': whole_le[i],
        "grammar": whole_gr[i],
        'omonID': whole_om[i],
        "root": whole_ro[i],
        "base": whole_ba[i],
        "variants": whole_va[i]
    }

In [117]:
len(whole_json.keys())

22842

In [121]:
with open('json_lemmas.json', 'w', encoding='utf-8') as f:
    json.dump(whole_json, f, ensure_ascii=False, indent=4)

In [143]:
whole_json_list = list(whole_json.values())

In [122]:
with open('json_lemmas_list.json', 'w', encoding='utf-8') as f:
    json.dump(list(whole_json.values()), f, ensure_ascii=False, indent=4)

In [55]:
flex = []
for xy in lemmas:
    flex.extend(re.findall('\([^\)]*\-([^\),/ ]+)[^\)]*\)', xy))
    
flex = list(set(flex))

In [56]:
flex

['иясул',
 'дал',
 "о'на",
 'ялдаса',
 'ун',
 'ина',
 'зулги',
 'лъулали',
 'налъул',
 'сулги',
 'зуд',
 'лги',
 'зазул',
 'малъул',
 'на',
 'лъулго',
 'яби',
 "лу'л",
 'идул',
 'и',
 "ро'л",
 "у'зул",
 'заби',
 'аса',
 'нал',
 'аби',
 'ял',
 'язул',
 'й',
 'мал',
 'э',
 'дерил',
 'ад',
 'инсул',
 'ве',
 "у'ялъул",
 "болъо'нитIанчIи",
 'дул',
 'одул',
 'олил',
 "цIаки'л",
 "шушби'",
 'лъ',
 'лали',
 'сулгияли',
 'уй',
 "а'зул",
 'са',
 'ил',
 'би',
 "за'л",
 'ялъул',
 'ой',
 "е'",
 'уял',
 "ма'л",
 "у'н",
 'хIелкал',
 'ухал',
 'а',
 'уялъул',
 "у'й",
 'ей',
 'н',
 'ан',
 'елъул',
 'лъулгияли',
 'го',
 'зулго',
 'ал',
 'йина',
 'ана',
 'йдул',
 'алзул',
 'бе',
 'ца',
 'ол',
 'ялъ',
 'зул',
 "сину'л",
 'асул',
 'уна',
 'зал',
 'аан',
 "и'л",
 "у'л",
 'ер',
 "а'санго",
 'алго',
 'иялъул',
 "би'",
 'нзаби',
 "е'р",
 "ду'л",
 'ани',
 'алъул',
 "а'л",
 'базул',
 'утул',
 'л',
 "я'л",
 'зулгияли',
 'лго',
 'ьзаби',
 "а'би",
 'сан',
 "а'",
 'вана',
 'ула',
 'алда',
 'агIи',
 'лъулги',
 'алъ',


In [57]:
j = 0
for i, xy in enumerate(lemmas):
    for fl in flex:
        if re.search(f'\([^\)]*[/, ]{fl}[,/ \)][^\)]*', xy):
            print(i+1)
            j+=1
print(j)

7598
7598
2


Работа с текстами

In [163]:
with open('avar_rus_paragraphs_clean.htm', 'r', encoding='utf-8') as f:
    pars = f.read().split('\n')
    articles = [a.split('</b>', maxsplit=1)[1].strip()[:-4] for a in pars]

#with open('avar_rus_articles.htm', 'w', encoding='utf-8') as fw:
#    fw.write('\n'.join(articles))
    
print(len(articles))

22842


In [258]:
with open('avar_rus_articles.htm', 'r', encoding='utf-8') as f:
    articles = f.read().split('\n')

In [107]:
print(len(articles))

22842


In [90]:
from collections import Counter

In [94]:
# check what italics we have
coun = Counter(re.findall('<i>([^>]+)</i>', '\n'.join(articles))).most_common()

In [95]:
coun

[('см.', 2527),
 ('род. п. от', 1627),
 ('масд. гл.', 1458),
 ('букв.', 1358),
 ('нареч.', 1203),
 ('масд. понуд. гл.', 971),
 ('диал.', 800),
 ('посл.', 797),
 ('перен.', 776),
 ('что-л.', 740),
 ('понуд.', 631),
 ('кого-л.', 564),
 ('мн. от', 551),
 ('разг.', 510),
 ('погов.', 390),
 ('собир.', 372),
 ('уст.', 362),
 ('учащ.', 345),
 ('масд. учащ. гл.', 344),
 ('понуд. от', 317),
 ('масд. учащ. понуд. гл.', 263),
 ('бот.', 253),
 ('фольк.', 248),
 ('и', 230),
 ('бран.', 214),
 ('чего-л.', 195),
 ('или', 193),
 ('редко', 180),
 ('кого-что-л.', 158),
 ('кому-л.', 155),
 ('числ.', 147),
 ('книжн.', 143),
 ('соотв.', 136),
 ('прич. от', 120),
 ('мн.', 118),
 ('понуд. от учащ.', 112),
 ('(имя собств. муж.)', 103),
 ('рел.', 101),
 ('от', 100),
 ('чем-л.', 92),
 ('грам.', 90),
 ('диал. см.', 77),
 ('нареч. см.', 74),
 ('в разн. знач.', 73),
 ('учащ. понуд.', 71),
 ('прост.', 70),
 ('с кем-л.', 70),
 ('т. е.', 69),
 ('ист.', 64),
 ('уст. книжн.', 63),
 ('мед.', 61),
 ('поэт.', 55),
 ('образ

In [ ]:
## IMPORTANT NOTE: don't forget that search engine MUST replace small l and 1 with an I, cause it's how many avars write it

In [97]:
cl = []
for a in articles:
    if a.startswith('<i>'):
        cl.append(re.match('<i>([^<]+)</i', a).group(1))
        
print(Counter(cl).most_common())

[('нареч.', 1124), ('масд. понуд. гл.', 925), ('масд. гл.', 723), ('понуд.', 608), ('диал.', 590), ('мн. от', 532), ('род. п. от', 476), ('собир.', 369), ('учащ.', 345), ('понуд. от', 316), ('см.', 294), ('уст.', 274), ('масд. учащ. понуд. гл.', 258), ('масд. учащ. гл.', 232), ('разг.', 217), ('бот.', 196), ('редко', 173), ('числ.', 144), ('мн.', 118), ('понуд. от учащ.', 110), ('книжн.', 106), ('диал. см.', 71), ('нареч. см.', 71), ('учащ. понуд.', 71), ('в разн. знач.', 70), ('уст. книжн.', 60), ('грам.', 48), ('межд.', 47), ('рел.', 47), ('мест.', 46), ('мед.', 46), ('звукоподр.', 46), ('регион.', 38), ('зоол.', 37), ('числ. порядк.', 36), ('диал. масд. гл.', 36), ('понуд. от учащ. гл.', 35), ('бран.', 28), ('абстр. имя от', 28), ('ист.', 27), ('дет.', 26), ('лит.', 24), ('анат.', 24), ('хим.', 23), ('разг. пренебр.', 22), ('мат.', 20), ('прост.', 19), ('диал. бот.', 18), ('масд. учащ.', 17), ('субст.', 17), ('не скл.', 15), ('муз.', 15), ('спорт.', 15), ('собир. см.', 14), ('воен.'

In [270]:
test_articles = articles[:100]

# translate, expl, examples, links

def working_article(art_str, lemma_dict, row_id):
    BASE = lemma_dict['base']
    ROOT = lemma_dict['root']
    
    if re.match("\d[\.\)]", art_str) or re.match("<i>[^>]+</i> \d[\.\)]", art_str):
        a_l = re.split('\d+[\.\)]', art_str)
        if a_l[0]:
            EXPLANATORY = re.search("<i>([^<]+)</i>", a_l[0]).group(1)
        else:
            EXPLANATORY = ""
        a_list = [a.strip() for a in a_l[1:]]
    else:
        a_list = [art_str.strip()]
        EXPLANATORY = ""
    
    output = []
    
    for a in a_list:
        _expl = ""
        a_link = []
        a_link_id = 0
        _tran = ""
        a_exam = []
        a_variants = []
        
        if '<' in a:
            
            if a.startswith('<i>'):
                italic, a_rest = a.split("</i>", maxsplit=1)
            
                if "см." in italic or a_rest.strip().startswith('<b>'):
                    _link = re.findall("<b>([^<]+)</b", a_rest)
                    if len(_link) < 3:
                    
                        a_link.extend(_link)
                    
                italic = re.sub('см\.', '', italic)
                italic = re.sub('<i>', '', italic)
                if italic.strip():
                    if EXPLANATORY: 
                        _expl = EXPLANATORY + "; " + italic.strip()
                    else:
                        _expl = italic.strip()
                a = a_rest.strip()
            
            # extract & delete links to other articles
            if re.search("см\.</i> ?<b>", a):
                _link = re.findall("см\.</i> ?<b>([^<]+)</b>", a)
                                    
                a_link.extend(_link)
                
                a = re.sub("<?i?>?см\.</i> ?<b>[^<]+</b>", "", a)
                
            
            # check if there are any examples
            if re.search("[◊;] <b>", a):
                raw_tran, very_raw_exams = re.split("[◊;] <b>", a, maxsplit=1)
                raw_exams = re.sub("^[◊;] ", '', very_raw_exams).split('<b>')
                #examples here
                raw_exams = [r_e.strip() for r_e in raw_exams]
                for ex in raw_exams:
                    ex = ex.replace("~ ", BASE + ' ')
                    if re.search('~([а-яI]+)', ex):
                        sugg = re.search('~([а-яI]+)', ex).group(1)
                        if sugg == ROOT[-len(sugg):] and len(sugg) > 2:
                            ROOT = ROOT[:-len(sugg)]
                    ex = ex.replace("~", ROOT)
                    try:
                        ex_avar, ex_rus = ex.split('</b>')
                    except:
                        print(f"{row_id+1}")
                    a_exam.append([ex_avar.strip(), ex_rus.strip()])
                #translate added (noch nicht bearbeitet)
                _tran = re.sub("<b>[^<]+</b>", '', raw_tran).strip()
            else:
                _a = re.sub("<b>[^<]+</b>", '', a)
                if _a.strip():
                    _tran = _a.strip()
            
        else:
            if re.search("[а-яА-Я]", a):
                _tran = a.strip()
        
        _tran = re.sub("</?[ib]>", '', _tran)
        
        # well, produce variants, i guess
        if _tran:
            _raw_var = re.sub("<i> ?и ?</i>", ',', _tran)
            _raw_var = re.sub("<i>[^<]+</i>", '', _raw_var)
            _raw_var = re.sub("<b>[^<]+</b>", '', _raw_var)
            _raw_var = re.sub(" \([^\)]+\)[ ,]", ' ', _raw_var)
            _raw_var = re.sub(" \([^\)]+\)$", ' ', _raw_var)
            _raw_var = re.sub(" {2,}", ' ', _raw_var)
            _raw_var_list = [x.strip() for x in re.split("[,;]", _raw_var)]
            _var_list = []
            for elem in _raw_var_list:
                elem = elem.strip()
                if re.match("\([а-яА-Я]+\)", elem):
                    _, v = elem.split(')', maxsplit=1)
                    _var_list.append(re.sub("[\(\)\!\?]", '', v).strip())
                _var_list.append(re.sub("[\(\)\!\?]", '', elem).strip())
            varLIST = []
            for elem in _var_list:
                if ' ' in elem:
                    mbjunks = elem.split(' ')
                    new_word = []
                    for mbjunk in mbjunks:
                        if '-' not in mbjunk:
                            new_word.append(mbjunk)
                    new_word = ' '.join(new_word).strip()
                    if re.search(' [а-я]{1,2}$', new_word):
                        new_word = new_word.rsplit(' ', maxsplit=1)[0]
                    varLIST.append(new_word)
                    
                
                else:
                    varLIST.append(elem)
                    
            a_variants = list(OrderedDict.fromkeys(varLIST))
            
            
        article_dict = {
            
            'translate': _tran, #string
            "examples": a_exam, #list of 2-elem-lists
            "explanation": _expl, #string
            "link": a_link, #list of 2-elem-lists: lemma and omonID
            "rus_vars": a_variants
        }
        output.append({"row_index": row_id, **lemma_dict, **article_dict})
        
    return output            

In [211]:
def article_list_working(article_list, id_list=[], verbose=0):
    
    if len(id_list) != len(article_list):
        id_list = [i for i in range(len(article_list))]
    
    output = []
    
    count = 0
    for index, article in enumerate(article_list):
        if verbose > 0:
            if count % verbose == 1:
                print(f"already done:\t{count-1}")
        art_list = working_article(article, whole_json_list[id_list[index]], id_list[index])
        output.extend(art_list)
    return output

In [245]:
test_json = article_list_working(test_articles, verbose=1)
with open("test_json_.json", 'w', encoding='utf-8') as f:
    json.dump(test_json, f, ensure_ascii=False, indent=4)

In [271]:
MEGA_JSON = article_list_working(articles, verbose=1000)
with open("mega_json_3000.json", 'w', encoding='utf-8') as f:
    json.dump(MEGA_JSON, f, ensure_ascii=False, indent=4)

In [272]:
import pandas as pd

In [280]:
df = pd.read_json("mega_json_3000.json", orient="records")
df['examples'][1]

[]

In [295]:
query = 'мяч'
answers = []

for index, row in df.iterrows():
    if query in row['rus_vars']:
        answers.append(index)
        
print(answers)

[22152, 22161, 29047]


In [296]:
df.loc[[22152, 22161, 29047]]

,row_index,lemma,grammar,omonID,root,base,variants,translate,examples,explanation,link,rus_vars
22152,16941,топ,"(-алъ, -алъул, -ал)",0,топ,топ,"[топ, топалъ, топалъул, топал]",мяч,"[[топ хIазе, играть в мяч]]",диал.,[торгIо'],[мяч]
22161,16949,торгIо',"(-ялъ//таргIи'ца, -ялъул, таргIи'л, таргIа'л)",0,торгIо,торгIо,"[торгIо, торгIоялъ, таргIица, торгIоялъул, тар...",мяч,"[[волейболалъул торгIо, волейбольный мяч;], [г...",,[],[мяч]
29047,21918,чIолгIо',"(-ялъ, -ялъул, -ял)",0,чIолгIо,чIолгIо,"[чIолгIо, чIолгIоялъ, чIолгIоялъул, чIолгIоял]",мяч,[],диал.,"[топ, торгIо']",[мяч]


In [299]:
df_raw_rus = df['rus_vars']

df_raw_rus

0                                 [а]
1         [иди отсюда, уходи, ступай]
2                    [это, этот, эта]
3        [вот этот, вот эта, вот это]
4                                  []
                     ...             
30200                            [эй]
30201                  [жизнь, житьё]
30202                          [яшма]
30203                          [ящик]
30204                          [ящур]
Name: rus_vars, Length: 30205, dtype: object

In [303]:
def sub_df_maker(df, t):
    df_sub = df[t]
    sub_dict = df_sub.to_dict()
    fut_keys = []
    fut_values = []
    for key in sub_dict:
        if sub_dict[key]:
            for word in sub_dict[key]:
                fut_keys.append(word)
                fut_values.append(key)
    df_res = pd.DataFrame(data={'word': fut_keys, 'big_ix': fut_values})
    return df_res
    
    


In [304]:
df_rus = sub_df_maker(df, 'rus_vars')
df_avar = sub_df_maker(df, 'variants')

In [323]:
df_rus.to_json('df_rus.json', orient='index', force_ascii=False, indent=4)
df_avar.to_json('df_avar.json', orient='index', force_ascii=False, indent=4)

In [340]:
first_letters = []
for x in df['base'].to_list():
    x = x.lower().strip('-')
    if re.match("[^кхглтчц]", x):
        first_letters.append(x[0])
    else:
        if re.match('[кхг]', x):
            if x[1] in "iъь":
                first_letters.append(x[:2])
            else:
                first_letters.append(x[0])
        elif re.match('[тцч]', x):
            if x[1] == "i":
                first_letters.append(x[:2])
            else:
                first_letters.append(x[0])
        else:
            if x[1] == "ъ":
                first_letters.append(x[:2])
            else:
                first_letters.append(x[0])

len(first_letters), set(first_letters)


(30205,
 {'а',
  'б',
  'в',
  'г',
  'гi',
  'гъ',
  'гь',
  'д',
  'е',
  'ж',
  'з',
  'и',
  'й',
  'к',
  'кi',
  'къ',
  'кь',
  'л',
  'лъ',
  'м',
  'н',
  'о',
  'п',
  'р',
  'с',
  'т',
  'тi',
  'у',
  'ф',
  'х',
  'хi',
  'хъ',
  'хь',
  'ц',
  'цi',
  'ч',
  'чi',
  'ш',
  'щ',
  'э',
  'ю',
  'я',
  'ё'})

In [ ]:
df_rus.to

In [274]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30205 entries, 0 to 30204
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   row_index    30205 non-null  int64 
 1   lemma        30205 non-null  object
 2   grammar      30205 non-null  object
 3   omonID       30205 non-null  int64 
 4   root         30205 non-null  object
 5   base         30205 non-null  object
 6   variants     30205 non-null  object
 7   translate    30205 non-null  object
 8   examples     30205 non-null  object
 9   explanation  30205 non-null  object
 10  link         30205 non-null  object
 11  rus_vars     30205 non-null  object
dtypes: int64(2), object(10)
memory usage: 2.8+ MB


In [101]:
## <i> ?и ?</i> -> ,


# LATER
# trying to add more grammar flexion
add_grm = {}
for i, art in enumerate(articles):
    grm = re.findall('\(\-[^\)]+\)', art)
    if grm:
        add_grm[i] = grm
        
#len(add_grm) = 215

add_grammar = {}
for i in add_grm:
    gr_str = '//'.join(add_grm[i])
    gr_str = re.sub("[\)\(]")
    pass

In [324]:
query = re.sub('[1\|l]', 'I', query)
query = re.sub('[^Iа-яА-Я\- ]', "", query).strip()

In [2]:
LET = ['а',
  'б',
  'в',
  'г',
  'гI',
  'гъ',
  'гь',
  'д',
  'е',
  'ё',
  'ж',
  'з',
  'и',
  'й',
  'к',
  'кI',
  'къ',
  'кь',
  'л',
  'лъ',
  'м',
  'н',
  'о',
  'п',
  'р',
  'с',
  'т',
  'тI',
  'у',
  'ф',
  'х',
  'хI',
  'хъ',
  'хь',
  'ц',
  'цI',
  'ч',
  'чI',
  'ш',
  'щ',
  'э',
  'ю',
  'я']

In [3]:
search_LET = ["__x__x" + x.lower() for x in LET]

In [4]:
search_LET

['__x__xа',
 '__x__xб',
 '__x__xв',
 '__x__xг',
 '__x__xгi',
 '__x__xгъ',
 '__x__xгь',
 '__x__xд',
 '__x__xе',
 '__x__xё',
 '__x__xж',
 '__x__xз',
 '__x__xи',
 '__x__xй',
 '__x__xк',
 '__x__xкi',
 '__x__xкъ',
 '__x__xкь',
 '__x__xл',
 '__x__xлъ',
 '__x__xм',
 '__x__xн',
 '__x__xо',
 '__x__xп',
 '__x__xр',
 '__x__xс',
 '__x__xт',
 '__x__xтi',
 '__x__xу',
 '__x__xф',
 '__x__xх',
 '__x__xхi',
 '__x__xхъ',
 '__x__xхь',
 '__x__xц',
 '__x__xцi',
 '__x__xч',
 '__x__xчi',
 '__x__xш',
 '__x__xщ',
 '__x__xэ',
 '__x__xю',
 '__x__xя']

In [4]:
len(LET)

43

In [2]:
import re
import json

In [13]:
with open("mega_json_3000_backup.json", 'r', encoding='utf-8') as f:
    pseudo_df = json.load(f)

In [5]:
possible_clitics = []

for record in pseudo_df:
    if record['lemma'].startswith('-') or record['base'].startswith('-') or record['base'].endswith('-') or record['lemma'].endswith('-'):
        possible_clitics.append([record['lemma'], record['base'], record['translate']])
        
for x in possible_clitics:
    print(x)

["-а'да", '-ада', 'тётя']
['-али', '-али', '-то']
["балу'гъ-", 'балугъ-', '(означает совершеннолетие, зрелость)']
["гванзу'-л гамалахъе!", 'гванзу-', 'к чертям']
['-ги', '-ги', 'и, да']
['-ги', '-ги', 'и  с последующими числами, а также со словом  в дробных числительных']
['-гури', '-гури', 'же, ведь']
['-гъун', '-гъун', 'вдоль']
['гьин-', 'гьин-', 'кровь, плоть в основе слов']
['-гIаги', '-гIаги', 'хоть, хотя, хотя бы']
["гIада'-", 'гIада-', 'голова в основе следующих слов:  (см.)']
['-гIан', '-гIан', 'также, как; как; словно, подобно; столько']
['-гIан', '-гIан', 'пока, чем и т. п.']
['-гIан', '-гIан', 'около, почти']
["-гIа'на(б)", '-гIанаб', 'с']
["-гIа'насе/(б)", '-гIанасеб', 'с']
["-гIа'насе/(б)", '-гIанасеб', 'около, приблизительно']
["-гIа'насе/(б)", '-гIанасеб', 'это происшествие не стоит того, чтобы о нём говорили']
["-даци'", '-даци', 'дядя (выступает вторым компонентом в сложных словах в сочетании собственных мужских имён)']
['-духъ', '-духъ', 'повидимому, чтобы было']
['-к

In [7]:
clitics_end = [
    'ги',
    'гури',
    'гъун',
    'гIаги',
    'гIан',
    'гIанаб',
    'гIанав',
    'гIанай',
    'гIанасеб',
    'гIанасев',
    'гIанасей',
    'духъ',
    'ккун',
    'кьераб',
    'лъила',
    'махIаб',
    'ни',
    'ниги',
    'тIалаяб',
    'уна',
    'хун',
    'хIа',
    'хIалаб',
    'цибилаб',
    'цин',
    'чи',
    'щинаб'
]
clitics_beg = [
    'балугъ',
    'гьин',
    'гIада',
    'нахъ'
]

In [14]:


pseudo_df[0].keys()

dict_keys(['row_index', 'lemma', 'grammar', 'omonID', 'root', 'base', 'variants', 'translate', 'examples', 'explanation', 'link', 'rus_vars'])

In [15]:
# big fix
new_records = []

for record in pseudo_df:
        
    # clean translations
    trans = record['translate']
    trans = trans.strip(";:, ")
    if not re.search('[а-яА-ЯI]', trans):
        trans = ''
    record['translate'] = trans
    
    # fixing rus vars a bit
    rus_vars = []
    rus_vars = [re.sub("'", '', x) for x in record['rus_vars']]
    rus_vars = [x for x in rus_vars if x]
    record['rus_vars'] = rus_vars
    
    # deleting empty avar vars
    if '' in record['variants']:
        avar_vars = [x for x in record['variants'] if x]
        record['variants'] = avar_vars
    
    new_records.append(record)

In [20]:
'aaaaavtrf'[:-3]

'aaaaav'

In [16]:
with open('mega_json_3000.json', 'w', encoding='utf-8') as f:
    json.dump(new_records, f, ensure_ascii=False, indent=4)